In [1]:
import tensorflow as tf 
import keras 
import keras.backend as K

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from collections import Counter

from keras import regularizers
from keras.models import Sequential, Model, load_model, model_from_json 
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Reshape, Concatenate,  Dropout 
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU

def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: float(majority/count) for cls, count in counter.items()}



class Estimator:
    l2p = 0.001
    @staticmethod
    def early_layers(inp, fm = (1,3), hid_act_func="relu"):
        # Start
        x = Conv2D(64, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # 1
        x = Conv2D(64, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)

        return x
    
    @staticmethod
    def late_layers(inp, num_classes, fm = (1,3), act_func="softmax", hid_act_func="relu", b_name="Identifier"):
        # 2
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # 3
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # End
        x = Flatten()(x)
        x = Dense(256, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(64, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_classes, activation=act_func, name = b_name)(x)

        return x
   
    @staticmethod
    def build(height, width, num_classes, name, fm = (1,3), act_func="softmax",hid_act_func="relu"):
        inp = Input(shape=(height, width, 1))
        early = Estimator.early_layers(inp, fm, hid_act_func=hid_act_func)
        late  = Estimator.late_layers(early, num_classes, fm, act_func=act_func, hid_act_func=hid_act_func)
        model = Model(inputs=inp, outputs=late ,name=name)
        return model

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt


def get_ds_infos():
    """
    Read the file includes data subject information.
    
    Data Columns:
    0: code [1-24]
    1: weight [kg]
    2: height [cm]
    3: age [years]
    4: gender [0:Female, 1:Male]
    
    Returns:
        A pandas DataFrame that contains inforamtion about data subjects' attributes 
    """ 

    dss = pd.read_csv("data_subjects_info.csv")
    print("[INFO] -- Data subjects' information is imported.")
    
    return dss

def set_data_types(data_types=["userAcceleration"]):
    """
    Select the sensors and the mode to shape the final dataset.
    
    Args:
        data_types: A list of sensor data type from this list: [attitude, gravity, rotationRate, userAcceleration] 

    Returns:
        It returns a list of columns to use for creating time-series from files.
    """
    dt_list = []
    for t in data_types:
        if t != "attitude":
            dt_list.append([t+".x",t+".y",t+".z"])
        else:
            dt_list.append([t+".roll", t+".pitch", t+".yaw"])

    return dt_list


def creat_time_series(dt_list, act_labels, trial_codes, mode="mag", labeled=True, combine_grav_acc=False):
    """
    Args:
        dt_list: A list of columns that shows the type of data we want.
        act_labels: list of activites
        trial_codes: list of trials
        mode: It can be "raw" which means you want raw data
        for every dimention of each data type,
        [attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)].
        or it can be "mag" which means you only want the magnitude for each data type: (x^2+y^2+z^2)^(1/2)
        labeled: True, if we want a labeld dataset. False, if we only want sensor values.
        combine_grav_acc: True, means adding each axis of gravity to  corresponding axis of userAcceleration.
    Returns: 
        It returns a time-series of sensor data.
    
    """
    num_data_cols = len(dt_list) if mode == "mag" else len(dt_list*3)

    if labeled:
        dataset = np.zeros((0,num_data_cols+7)) # "7" --> [act, code, weight, height, age, gender, trial] 
    else:
        dataset = np.zeros((0,num_data_cols))
        
    ds_list = get_ds_infos()
    
    print("[INFO] -- Creating Time-Series")
    for sub_id in ds_list["code"]:
        for act_id, act in enumerate(act_labels):
            for trial in trial_codes[act_id]:
                fname = 'A_DeviceMotion_data/'+act+'_'+str(trial)+'/sub_'+str(int(sub_id))+'.csv'
                raw_data = pd.read_csv(fname)
                raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
                vals = np.zeros((len(raw_data), num_data_cols))
                
                if combine_grav_acc:
                    raw_data["userAcceleration.x"] = raw_data["userAcceleration.x"].add(raw_data["gravity.x"])
                    raw_data["userAcceleration.y"] = raw_data["userAcceleration.y"].add(raw_data["gravity.y"])
                    raw_data["userAcceleration.z"] = raw_data["userAcceleration.z"].add(raw_data["gravity.z"])
                
                for x_id, axes in enumerate(dt_list):
                    if mode == "mag":
                        vals[:,x_id] = (raw_data[axes]**2).sum(axis=1)**0.5        
                    else:
                        vals[:,x_id*3:(x_id+1)*3] = raw_data[axes].values
                    vals = vals[:,:num_data_cols]
                if labeled:
                    lbls = np.array([[act_id,
                            sub_id-1,
                            ds_list["weight"][sub_id-1],
                            ds_list["height"][sub_id-1],
                            ds_list["age"][sub_id-1],
                            ds_list["gender"][sub_id-1],
                            trial          
                           ]]*len(raw_data))
                    vals = np.concatenate((vals, lbls), axis=1)
                dataset = np.append(dataset,vals, axis=0)
    cols = []
    for axes in dt_list:
        if mode == "raw":
            cols += axes
        else:
            cols += [str(axes[0][:-2])]
            
    if labeled:
        cols += ["act", "id", "weight", "height", "age", "gender", "trial"]
    
    dataset = pd.DataFrame(data=dataset, columns=cols)
    return dataset
#________________________________
#________________________________

def ts_to_secs(dataset, w, s, standardize = False, **options):
    
    data = dataset[dataset.columns[:-7]].values    
    act_labels = dataset["act"].values
    id_labels = dataset["id"].values
    trial_labels = dataset["trial"].values

    mean = 0
    std = 1
    if standardize:
        ## Standardize each sensor’s data to have a zero mean and unity standard deviation.
        ## As usual, we normalize test dataset by training dataset's parameters 
        if options:
            mean = options.get("mean")
            std = options.get("std")
            print("[INFO] -- Test/Val Data has been standardized")
        else:
            mean = data.mean(axis=0)
            std = data.std(axis=0)
            print("[INFO] -- Training Data has been standardized: the mean is = "+str(mean)+" ; and the std is = "+str(std))            

        data -= mean
        data /= std
    else:
        print("[INFO] -- Without Standardization.....")

    ## We want the Rows of matrices show each Feature and the Columns show time points.
    data = data.T

    m = data.shape[0]   # Data Dimension 
    ttp = data.shape[1] # Total Time Points
    number_of_secs = int(round(((ttp - w)/s)))

    ##  Create a 3D matrix for Storing Sections  
    secs_data = np.zeros((number_of_secs , m , w ))
    act_secs_labels = np.zeros(number_of_secs)
    id_secs_labels = np.zeros(number_of_secs)

    k=0
    for i in range(0 , ttp-w, s):
        j = i // s
        if j >= number_of_secs:
            break
        if id_labels[i] != id_labels[i+w-1]: 
            continue
        if act_labels[i] != act_labels[i+w-1]: 
            continue
        if trial_labels[i] != trial_labels[i+w-1]:
            continue
            
        secs_data[k] = data[:, i:i+w]
        act_secs_labels[k] = act_labels[i].astype(int)
        id_secs_labels[k] = id_labels[i].astype(int)
        k = k+1
        
    secs_data = secs_data[0:k]
    act_secs_labels = act_secs_labels[0:k]
    id_secs_labels = id_secs_labels[0:k]
    return secs_data, act_secs_labels, id_secs_labels, mean, std
##________________________________________________________________


ACT_LABELS = ["dws","ups", "wlk", "jog", "std", "sit"]
TRIAL_CODES = {
    ACT_LABELS[0]:[1,2,11],
    ACT_LABELS[1]:[3,4,12],
    ACT_LABELS[2]:[7,8,15],
    ACT_LABELS[3]:[9,16],
    ACT_LABELS[4]:[6,14],
    ACT_LABELS[5]:[5,13],
}

In [3]:
#https://stackoverflow.com/a/45305384/5210098
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def eval_id(sdt, mode, ep, cga):

    print("[INFO] -- Selected sensor data types: "+str(sdt)+" -- Mode: "+str(mode)+" -- Grav+Acc: "+str(cga))    
    act_labels = ACT_LABELS [0:4]

    print("[INFO] -- Selected activites: "+str(act_labels))    
    trial_codes = [TRIAL_CODES[act] for act in act_labels]
    dt_list = set_data_types(sdt)
    dataset = creat_time_series(dt_list, act_labels, trial_codes, mode=mode, labeled=True, combine_grav_acc = cga)
    print("[INFO] -- Shape of time-Series dataset:"+str(dataset.shape))    


    #*****************
    TRAIN_TEST_TYPE = "trial" # "subject" or "trial"
    #*****************

    if TRAIN_TEST_TYPE == "subject":
        test_ids = [4,9,11,21]
        print("[INFO] -- Test IDs: "+str(test_ids))
        test_ts = dataset.loc[(dataset['id'].isin(test_ids))]
        train_ts = dataset.loc[~(dataset['id'].isin(test_ids))]
    else:
        test_trail = [11,12,13,14,15,16]  
        print("[INFO] -- Test Trials: "+str(test_trail))
        test_ts = dataset.loc[(dataset['trial'].isin(test_trail))]
        train_ts = dataset.loc[~(dataset['trial'].isin(test_trail))]

    print("[INFO] -- Shape of Train Time-Series :"+str(train_ts.shape))
    print("[INFO] -- Shape of Test Time-Series :"+str(test_ts.shape))
    
    print("___________________________________________________")

    ## This Variable Defines the Size of Sliding Window
    ## ( e.g. 100 means in each snapshot we just consider 100 consecutive observations of each sensor) 
    w = 128 # 50 Equals to 1 second for MotionSense Dataset (it is on 50Hz samplig rate)
    ## Here We Choose Step Size for Building Diffrent Snapshots from Time-Series Data
    ## ( smaller step size will increase the amount of the instances and higher computational cost may be incurred )
    s = 10
    train_data, act_train, id_train, train_mean, train_std = ts_to_secs(train_ts.copy(),
                                                                       w,
                                                                       s,
                                                                       standardize = True)
    
   
    s = 10
    test_data, act_test, id_test, test_mean, test_std = ts_to_secs(test_ts.copy(),
                                                                  w,
                                                                  s,
                                                                  standardize = True,
                                                                  mean = train_mean, 
                                                                  std = train_std)
    
    print("[INFO] -- Training Sections: "+str(train_data.shape))
    print("[INFO] -- Test Sections:  "+str(test_data.shape))


    id_train_labels = to_categorical(id_train)
    id_test_labels = to_categorical(id_test)
    
    act_train_labels = to_categorical(act_train)
    act_test_labels = to_categorical(act_test)
    
    ## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
    train_data = np.expand_dims(train_data,axis=3)
    print("[INFO] -- Training Sections:"+str(train_data.shape))

    test_data = np.expand_dims(test_data,axis=3)
    print("[INFO] -- Test Sections:"+str(test_data.shape))

    height = train_data.shape[1]
    width = train_data.shape[2]

    id_class_numbers = 24
    act_class_numbers = 4
    fm = (1,5)

    print("___________________________________________________")
    ## Callbacks
    #eval_metric= "val_acc"
    eval_metric= "val_f1_metric"    
    early_stop = keras.callbacks.EarlyStopping(monitor=eval_metric, mode='max', patience = 7)
    filepath="MID.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor=eval_metric, verbose=0, save_best_only=True, mode='max')
    callbacks_list = [early_stop,
                      checkpoint
                     ]
    ## Callbacks
    eval_id = Estimator.build(height, width, id_class_numbers, name ="EVAL_ID", fm=fm, act_func="softmax",hid_act_func="relu")
    eval_id.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc', f1_metric])
    print("Model Size = "+str(eval_id.count_params()))

    eval_id.fit(train_data, id_train_labels,
                validation_data = (test_data, id_test_labels),
                epochs = ep,
                batch_size = 128,
                verbose = 0,
                class_weight = get_class_weights(np.argmax(id_train_labels,axis=1)),
                callbacks = callbacks_list
               )

    eval_id.load_weights("MID.best.hdf5")
    eval_id.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc',f1_metric])

    result1 = eval_id.evaluate(test_data, id_test_labels, verbose = 2)
    id_acc = result1[1]
    print("***[RESULT]*** ID Accuracy: "+str(id_acc))
    rf1 = result1[2].round(4)*100
    print("***[RESULT]*** ID F1: "+str(rf1))
    
    preds = eval_id.predict(test_data)
    preds = np.argmax(preds, axis=1)
    conf_mat = confusion_matrix(np.argmax(id_test_labels, axis=1), preds)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print("***[RESULT]*** ID  Confusion Matrix")
    print((np.array(conf_mat).diagonal()).round(3)*100)  
    
    d_test_ids = [4,9,11,21]
    to_avg = 0
    for i in range(len(d_test_ids)):
        true_positive = conf_mat[d_test_ids[i],d_test_ids[i]]
        print("True Positive Rate for "+str(d_test_ids[i])+" : "+str(true_positive*100))
        to_avg+=true_positive
    atp = to_avg/len(d_test_ids)    
    print("Average TP:"+str(atp*100))    
    
    f1id = f1_score(np.argmax(id_test_labels, axis=1), preds, average=None).mean()
    print("***[RESULT]*** ID Averaged F-1 Score : "+str(f1id))
    
    return [round(id_acc,4), round(f1id,4), round(atp,4)]

In [4]:
results ={}

In [5]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["rotationRate"]
mode = "mag"
ep = 40
cga = False # Add gravity to acceleration or not
for i in range(5):
    results[str(sdt)+"--"+str(mode)+"--"+str(cga)+"--"+str(i)] = eval_id(sdt, mode, ep, cga)

[INFO] -- Selected sensor data types: ['rotationRate'] -- Mode: mag -- Grav+Acc: False
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 8)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 8)
[INFO] -- Shape of Test Time-Series :(145687, 8)
___________________________________________________
[INFO] -- Training Data has been standardized: the mean is = [2.20896278] ; and the std is = [1.42146386]
[INFO] -- Test/Val Data has been standardized
[INFO] -- Training Sections: (60059, 1, 128)
[INFO] -- Test Sections:  (13344, 1, 128)
[INFO] -- Training Sections:(60059, 1, 128, 1)
[INFO] -- Test Sections:(13344, 1, 128, 1)
___________________________________________________
Model Size = 122200
***[RESULT]*** ID Accuracy: 0.8687799760191847
***[RESULT]*** ID F1: 86.37
***[RESULT]*** ID  Confusion Matrix
[71.3 99.4 

In [6]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["rotationRate"]
mode = "raw"
ep = 40
cga = False # Add gravity to acceleration or not
for i in range(5):
    results[str(sdt)+"--"+str(mode)+"--"+str(cga)+"--"+str(i)] = eval_id(sdt, mode, ep, cga)

[INFO] -- Selected sensor data types: ['rotationRate'] -- Mode: raw -- Grav+Acc: False
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 10)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 10)
[INFO] -- Shape of Test Time-Series :(145687, 10)
___________________________________________________
[INFO] -- Training Data has been standardized: the mean is = [0.00676741 0.02878308 0.02359966] ; and the std is = [1.74135109 1.64053436 1.08396877]
[INFO] -- Test/Val Data has been standardized
[INFO] -- Training Sections: (60059, 3, 128)
[INFO] -- Test Sections:  (13344, 3, 128)
[INFO] -- Training Sections:(60059, 3, 128, 1)
[INFO] -- Test Sections:(13344, 3, 128, 1)
___________________________________________________
Model Size = 253272
***[RESULT]*** ID Accuracy: 0.9426708633093526
***[RESULT]*** ID F1: 94.23


In [7]:
results

{"['rotationRate']--mag--False--0": [0.8688, 0.8661, 0.8968],
 "['rotationRate']--mag--False--1": [0.8429, 0.8377, 0.8979],
 "['rotationRate']--mag--False--2": [0.8563, 0.8521, 0.8591],
 "['rotationRate']--mag--False--3": [0.8696, 0.8618, 0.9192],
 "['rotationRate']--mag--False--4": [0.8554, 0.8522, 0.9038],
 "['rotationRate']--raw--False--0": [0.9427, 0.9424, 0.978],
 "['rotationRate']--raw--False--1": [0.946, 0.9445, 0.9879],
 "['rotationRate']--raw--False--2": [0.9251, 0.9246, 0.9713],
 "['rotationRate']--raw--False--3": [0.9344, 0.9325, 0.9817],
 "['rotationRate']--raw--False--4": [0.9327, 0.9325, 0.9849]}

In [8]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["userAcceleration"]
mode = "mag"
ep = 40
cga = True # Add gravity to acceleration or not
for i in range(5):
    results[str(sdt)+"--"+str(mode)+"--"+str(cga)+"--"+str(i)] = eval_id(sdt, mode, ep, cga)

[INFO] -- Selected sensor data types: ['userAcceleration'] -- Mode: mag -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 8)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 8)
[INFO] -- Shape of Test Time-Series :(145687, 8)
___________________________________________________
[INFO] -- Training Data has been standardized: the mean is = [1.19815844] ; and the std is = [0.70139403]
[INFO] -- Test/Val Data has been standardized
[INFO] -- Training Sections: (60059, 1, 128)
[INFO] -- Test Sections:  (13344, 1, 128)
[INFO] -- Training Sections:(60059, 1, 128, 1)
[INFO] -- Test Sections:(13344, 1, 128, 1)
___________________________________________________
Model Size = 122200
***[RESULT]*** ID Accuracy: 0.8779976019184652
***[RESULT]*** ID F1: 87.22
***[RESULT]*** ID  Confusion Matrix
[76.3 93

In [9]:
results

{"['userAcceleration']--mag--True--0": [0.878, 0.8633, 0.9027],
 "['userAcceleration']--mag--True--1": [0.8872, 0.8761, 0.8893],
 "['userAcceleration']--mag--True--2": [0.8864, 0.8777, 0.8954],
 "['userAcceleration']--mag--True--3": [0.8786, 0.8715, 0.8811],
 "['userAcceleration']--mag--True--4": [0.8829, 0.8676, 0.9006]}

In [10]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["userAcceleration"]
mode = "raw"
ep = 40
cga = True # Add gravity to acceleration or not
for i in range(5):
    results[str(sdt)+"--"+str(mode)+"--"+str(cga)+"--"+str(i)] = eval_id(sdt, mode, ep, cga)

[INFO] -- Selected sensor data types: ['userAcceleration'] -- Mode: raw -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 10)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 10)
[INFO] -- Shape of Test Time-Series :(145687, 10)
___________________________________________________
[INFO] -- Training Data has been standardized: the mean is = [ 0.02367904  0.95806826 -0.05104623] ; and the std is = [0.48401853 0.74077811 0.47270486]
[INFO] -- Test/Val Data has been standardized
[INFO] -- Training Sections: (60059, 3, 128)
[INFO] -- Test Sections:  (13344, 3, 128)
[INFO] -- Training Sections:(60059, 3, 128, 1)
[INFO] -- Test Sections:(13344, 3, 128, 1)
___________________________________________________
Model Size = 253272
***[RESULT]*** ID Accuracy: 0.9224370503597122
***[RESULT]*** ID F1: 

In [11]:
results 

{"['userAcceleration']--mag--True--0": [0.878, 0.8633, 0.9027],
 "['userAcceleration']--mag--True--1": [0.8872, 0.8761, 0.8893],
 "['userAcceleration']--mag--True--2": [0.8864, 0.8777, 0.8954],
 "['userAcceleration']--mag--True--3": [0.8786, 0.8715, 0.8811],
 "['userAcceleration']--mag--True--4": [0.8829, 0.8676, 0.9006],
 "['userAcceleration']--raw--True--0": [0.9224, 0.9198, 0.9822],
 "['userAcceleration']--raw--True--1": [0.9288, 0.9296, 0.9888],
 "['userAcceleration']--raw--True--2": [0.9277, 0.9244, 0.9796],
 "['userAcceleration']--raw--True--3": [0.9258, 0.924, 0.9831],
 "['userAcceleration']--raw--True--4": [0.9275, 0.9255, 0.992]}

In [12]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["rotationRate","userAcceleration"]
mode = "mag"
ep = 40
cga = True # Add gravity to acceleration or not
for i in range(5):
    results[str(sdt)+"--"+str(mode)+"--"+str(cga)+"--"+str(i)] = eval_id(sdt, mode, ep, cga)

[INFO] -- Selected sensor data types: ['rotationRate', 'userAcceleration'] -- Mode: mag -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 9)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 9)
[INFO] -- Shape of Test Time-Series :(145687, 9)
___________________________________________________
[INFO] -- Training Data has been standardized: the mean is = [2.20896278 1.19815844] ; and the std is = [1.42146386 0.70139403]
[INFO] -- Test/Val Data has been standardized
[INFO] -- Training Sections: (60059, 2, 128)
[INFO] -- Test Sections:  (13344, 2, 128)
[INFO] -- Training Sections:(60059, 2, 128, 1)
[INFO] -- Test Sections:(13344, 2, 128, 1)
___________________________________________________
Model Size = 187736
***[RESULT]*** ID Accuracy: 0.9230365707434053
***[RESULT]*** ID F1: 92.179999999

In [13]:
results

{"['rotationRate', 'userAcceleration']--mag--True--0": [0.923, 0.9114, 0.9617],
 "['rotationRate', 'userAcceleration']--mag--True--1": [0.9314,
  0.9242,
  0.9616],
 "['rotationRate', 'userAcceleration']--mag--True--2": [0.9392, 0.9339, 0.949],
 "['rotationRate', 'userAcceleration']--mag--True--3": [0.9389,
  0.9332,
  0.9584],
 "['rotationRate', 'userAcceleration']--mag--True--4": [0.9327,
  0.9262,
  0.9621],
 "['userAcceleration']--mag--True--0": [0.878, 0.8633, 0.9027],
 "['userAcceleration']--mag--True--1": [0.8872, 0.8761, 0.8893],
 "['userAcceleration']--mag--True--2": [0.8864, 0.8777, 0.8954],
 "['userAcceleration']--mag--True--3": [0.8786, 0.8715, 0.8811],
 "['userAcceleration']--mag--True--4": [0.8829, 0.8676, 0.9006],
 "['userAcceleration']--raw--True--0": [0.9224, 0.9198, 0.9822],
 "['userAcceleration']--raw--True--1": [0.9288, 0.9296, 0.9888],
 "['userAcceleration']--raw--True--2": [0.9277, 0.9244, 0.9796],
 "['userAcceleration']--raw--True--3": [0.9258, 0.924, 0.9831],
 "

In [14]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["rotationRate","userAcceleration"]
mode = "raw"
ep = 40
cga = True # Add gravity to acceleration or not
for i in range(5):
    results[str(sdt)+"--"+str(mode)+"--"+str(cga)+"--"+str(i)] = eval_id(sdt, mode, ep, cga)

[INFO] -- Selected sensor data types: ['rotationRate', 'userAcceleration'] -- Mode: raw -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 13)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 13)
[INFO] -- Shape of Test Time-Series :(145687, 13)
___________________________________________________
[INFO] -- Training Data has been standardized: the mean is = [ 0.00676741  0.02878308  0.02359966  0.02367904  0.95806826 -0.05104623] ; and the std is = [1.74135109 1.64053436 1.08396877 0.48401853 0.74077811 0.47270486]
[INFO] -- Test/Val Data has been standardized
[INFO] -- Training Sections: (60059, 6, 128)
[INFO] -- Test Sections:  (13344, 6, 128)
[INFO] -- Training Sections:(60059, 6, 128, 1)
[INFO] -- Test Sections:(13344, 6, 128, 1)
___________________________________________________
Mode

In [15]:
results

{"['rotationRate', 'userAcceleration']--mag--True--0": [0.923, 0.9114, 0.9617],
 "['rotationRate', 'userAcceleration']--mag--True--1": [0.9314,
  0.9242,
  0.9616],
 "['rotationRate', 'userAcceleration']--mag--True--2": [0.9392, 0.9339, 0.949],
 "['rotationRate', 'userAcceleration']--mag--True--3": [0.9389,
  0.9332,
  0.9584],
 "['rotationRate', 'userAcceleration']--mag--True--4": [0.9327,
  0.9262,
  0.9621],
 "['rotationRate', 'userAcceleration']--raw--True--0": [0.935, 0.9325, 0.9889],
 "['rotationRate', 'userAcceleration']--raw--True--1": [0.9302, 0.929, 0.9914],
 "['rotationRate', 'userAcceleration']--raw--True--2": [0.9289,
  0.9292,
  0.9897],
 "['rotationRate', 'userAcceleration']--raw--True--3": [0.9248,
  0.9188,
  0.9752],
 "['rotationRate', 'userAcceleration']--raw--True--4": [0.9426,
  0.9418,
  0.9864],
 "['userAcceleration']--mag--True--0": [0.878, 0.8633, 0.9027],
 "['userAcceleration']--mag--True--1": [0.8872, 0.8761, 0.8893],
 "['userAcceleration']--mag--True--2": [0

In [ ]:
#https://stackoverflow.com/a/45305384/5210098
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def eval_id(sdt, mode, ep, cga):

    print("[INFO] -- Selected sensor data types: "+str(sdt)+" -- Mode: "+str(mode)+" -- Grav+Acc: "+str(cga))    
    act_labels = ACT_LABELS [0:4]

    print("[INFO] -- Selected activites: "+str(act_labels))    
    trial_codes = [TRIAL_CODES[act] for act in act_labels]
    dt_list = set_data_types(sdt)
    dataset = creat_time_series(dt_list, act_labels, trial_codes, mode=mode, labeled=True, combine_grav_acc = cga)
    print("[INFO] -- Shape of time-Series dataset:"+str(dataset.shape))    


    #*****************
    TRAIN_TEST_TYPE = "trial" # "subject" or "trial"
    #*****************

    if TRAIN_TEST_TYPE == "subject":
        test_ids = [4,9,11,21]
        print("[INFO] -- Test IDs: "+str(test_ids))
        test_ts = dataset.loc[(dataset['id'].isin(test_ids))]
        train_ts = dataset.loc[~(dataset['id'].isin(test_ids))]
    else:
        test_trail = [11,12,13,14,15,16]  
        print("[INFO] -- Test Trials: "+str(test_trail))
        test_ts = dataset.loc[(dataset['trial'].isin(test_trail))]
        train_ts = dataset.loc[~(dataset['trial'].isin(test_trail))]

    print("[INFO] -- Shape of Train Time-Series :"+str(train_ts.shape))
    print("[INFO] -- Shape of Test Time-Series :"+str(test_ts.shape))
    
#     print("___________Train_VAL____________")
#     val_trail = [11,12,13,14,15,16]
#     val_ts = train_ts.loc[(train_ts['trial'].isin(val_trail))]
#     train_ts = train_ts.loc[~(train_ts['trial'].isin(val_trail))]
#     print("[INFO] -- Training Time-Series :"+str(train_ts.shape))
#     print("[INFO] -- Validation Time-Series :"+str(val_ts.shape)) 
    print("___________________________________________________")

    ## This Variable Defines the Size of Sliding Window
    ## ( e.g. 100 means in each snapshot we just consider 100 consecutive observations of each sensor) 
    w = 128 # 50 Equals to 1 second for MotionSense Dataset (it is on 50Hz samplig rate)
    ## Here We Choose Step Size for Building Diffrent Snapshots from Time-Series Data
    ## ( smaller step size will increase the amount of the instances and higher computational cost may be incurred )
    s = 10
    train_data, act_train, id_train, train_mean, train_std = ts_to_secs(train_ts.copy(),
                                                                       w,
                                                                       s,
                                                                       standardize = True)
    
   
    s = 10
    test_data, act_test, id_test, test_mean, test_std = ts_to_secs(test_ts.copy(),
                                                                  w,
                                                                  s,
                                                                  standardize = True,
                                                                  mean = train_mean, 
                                                                  std = train_std)
    
    print("[INFO] -- Training Sections: "+str(train_data.shape))
    print("[INFO] -- Test Sections:  "+str(test_data.shape))


    id_train_labels = to_categorical(id_train)
    id_test_labels = to_categorical(id_test)
    
    act_train_labels = to_categorical(act_train)
    act_test_labels = to_categorical(act_test)
    
    ## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
    train_data = np.expand_dims(train_data,axis=3)
    print("[INFO] -- Training Sections:"+str(train_data.shape))

    test_data = np.expand_dims(test_data,axis=3)
    print("[INFO] -- Test Sections:"+str(test_data.shape))

    height = train_data.shape[1]
    width = train_data.shape[2]

    id_class_numbers = 24
    act_class_numbers = 4
    fm = (2,5)

    print("___________________________________________________")
    ## Callbacks
    #eval_metric= "val_acc"
    eval_metric= "val_f1_metric"    
    early_stop = keras.callbacks.EarlyStopping(monitor=eval_metric, mode='max', patience = 7)
    filepath="MID.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor=eval_metric, verbose=0, save_best_only=True, mode='max')
    callbacks_list = [early_stop,
                      checkpoint
                     ]
    ## Callbacks
    eval_id = Estimator.build(height, width, id_class_numbers, name ="EVAL_ID", fm=fm, act_func="softmax",hid_act_func="relu")
    eval_id.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc', f1_metric])
    print("Model Size = "+str(eval_id.count_params()))

    eval_id.fit(train_data, id_train_labels,
                validation_data = (test_data, id_test_labels),
                epochs = ep,
                batch_size = 128,
                verbose = 0,
                class_weight = get_class_weights(np.argmax(id_train_labels,axis=1)),
                callbacks = callbacks_list
               )

    eval_id.load_weights("MID.best.hdf5")
    eval_id.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc',f1_metric])

    result1 = eval_id.evaluate(test_data, id_test_labels, verbose = 2)
    id_acc = result1[1]
    print("***[RESULT]*** ID Accuracy: "+str(id_acc))
    rf1 = result1[2].round(4)*100
    print("***[RESULT]*** ID F1: "+str(rf1))
    
    preds = eval_id.predict(test_data)
    preds = np.argmax(preds, axis=1)
    conf_mat = confusion_matrix(np.argmax(id_test_labels, axis=1), preds)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print("***[RESULT]*** ID  Confusion Matrix")
    print((np.array(conf_mat).diagonal()).round(3)*100)  
    
    d_test_ids = [4,9,11,21]
    to_avg = 0
    for i in range(len(d_test_ids)):
        true_positive = conf_mat[d_test_ids[i],d_test_ids[i]]
        print("True Positive Rate for "+str(d_test_ids[i])+" : "+str(true_positive*100))
        to_avg+=true_positive
    atp = to_avg/len(d_test_ids)    
    print("Average TP:"+str(atp*100))    
    
    f1id = f1_score(np.argmax(id_test_labels, axis=1), preds, average=None).mean()
    print("***[RESULT]*** ID Averaged F-1 Score : "+str(f1id))
    
    return [round(id_acc,4), round(f1id,4), round(atp,4)]

In [ ]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["rotationRate","userAcceleration"]
mode = "mag"
ep = 40
cga = True # Add gravity to acceleration or not
for i in range(5):
    results[str(sdt)+"-2D-"+str(mode)+"--"+str(cga)+"--"+str(i)] = eval_id(sdt, mode, ep, cga)

[INFO] -- Selected sensor data types: ['rotationRate', 'userAcceleration'] -- Mode: mag -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 9)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 9)
[INFO] -- Shape of Test Time-Series :(145687, 9)
___________________________________________________
[INFO] -- Training Data has been standardized: the mean is = [2.20896278 1.19815844] ; and the std is = [1.42146386 0.70139403]
[INFO] -- Test/Val Data has been standardized
[INFO] -- Training Sections: (60059, 2, 128)
[INFO] -- Test Sections:  (13344, 2, 128)
[INFO] -- Training Sections:(60059, 2, 128, 1)
[INFO] -- Test Sections:(13344, 2, 128, 1)
___________________________________________________
Model Size = 223896
